In [1]:
from astropy.io import fits
import numpy as np
from astropy.table import Table
from decimal import *
import pandas as pd
from math import *
from astropy.coordinates import SkyCoord
import astropy.units as u
from astropy.io import ascii
import glob
import os
from pathlib import Path
from astropy.wcs import WCS
from astropy.utils.data import get_pkg_data_filename
import matplotlib.pyplot as plt
from astropy.nddata import Cutout2D
plt.style.use("stylesheet.txt")
from functools import reduce
import multiwavlength_fluxes as mf

In [2]:
ms_cdfs = ["CDFS-18", "CDFS-19"]
ms_es1 = ["ES1-12","ES1-18","ES1-25","ES1-26","ES1-34","ES1-35"]
ms_xmm = ["XMM-11","XMM-19","XMM-27","XMM-29","XMM-30"]
ms_bcgs = ms_cdfs + ms_es1 + ms_xmm
sb_es1_bcgs = ["ES1_z_0.88", "ES1_z_0.99", "ES1_z_0.99b", "ES1_z_1.04", "ES1_z_1.38", "ES1_z_1.40", "ES1_z_1.60", "ES1_z_1.65", "ES1_z_1.70"]
sb_xmm_bcgs = ["XMM_113", "XMM_z_0.9", "XMM_z_1.0", "XMM_z_0.81"]
sb_bcgs = sb_es1_bcgs + sb_xmm_bcgs
all_bcgs = ms_bcgs + sb_bcgs

In [3]:
for bcg in all_bcgs:
    files = glob.glob(bcg + "/*.reg")
    for file in files:
        os.remove(file)

In [4]:
bcg_coords = pd.read_csv("BCG_coords.txt", delim_whitespace = True, names = ['id', 'ra', 'dec']).sort_values('id', ascending = True).reset_index(drop = True)

In [5]:
bcg_info = pd.read_csv("BCG_info.csv")[["BCG","ALMA flux", "Flux_err"]]
final_flux_catalog = pd.DataFrame()

In [6]:
for ind, row in bcg_coords.iterrows():
    des_catalog = mf.des_flux(row['id'], row['ra'], row['dec'])
    irac_catalog = mf.irac_flux(row['id'], row['ra'], row['dec'])
    hermes_catalog = mf.hermes_flux(row['id'], row['ra'], row['dec'])
    hermes_catalog['spitzer.mips.24'] = hermes_catalog['spitzer.mips.24']*0.001
    hermes_catalog['spitzer.mips.24_err'] = hermes_catalog['spitzer.mips.24_err']*0.001
    des_irac = pd.merge(des_catalog, irac_catalog, on = 'id')
    all_fluxes = pd.merge(des_irac, hermes_catalog, on = 'id')
    final_flux_catalog = final_flux_catalog.append(all_fluxes)

In [7]:
final_flux_catalog = final_flux_catalog.reset_index(drop = True)
bcg_info = bcg_info.rename(columns = {"BCG":"id"})
all_fluxes = pd.merge(final_flux_catalog,bcg_info,  on = 'id')
all_fluxes['redshift'] = [0.80758, 0.69484, 0.74744, 0.8068, 1.08807, 0.85192, 1.7, 1.7, 0.56395, 1.19372, 0.91886, 1.04, 1.38, 1.4, 1.6, 1.65, 0.91651, 0.79, 
                         1.04789, 1.38, 1.45, 1.45, 1.6, 0.7827207, 0.84957, 0.5352355]

In [8]:
all_fluxes = all_fluxes[['id','redshift', 'des_g', 'des_r', 'des_i', 'des_z', 'des_Y', 'des_g_err', 'des_r_err',
       'des_i_err', 'des_z_err', 'des_Y_err', 'spitzer.irac.ch1',
       'spitzer.irac.ch1_err', 'spitzer.irac.ch2', 'spitzer.irac.ch2_err',
       'spitzer.irac.ch3', 'spitzer.irac.ch3_err', 'spitzer.irac.ch4',
       'spitzer.irac.ch4_err', 'spitzer.mips.24', 'spitzer.mips.24_err',
       'herschel.spire.PSW', 'herschel.spire.PSW_err', 'herschel.spire.PMW',
       'herschel.spire.PMW_err', 'herschel.spire.PLW',
       'herschel.spire.PLW_err', 'ALMA flux', 'Flux_err']]

In [9]:
high_z_es1 = ['ES1_z_1.38', 'ES1_z_1.40', 'ES1_z_1.60',
       'ES1_z_1.65', 'ES1_z_1.70']

In [10]:
path = "/Users/arames52/Downloads/cigale-v2020.0/pcigale/data/"

In [11]:
for bcg in high_z_es1:
    flux_table = all_fluxes[all_fluxes['id'] == bcg]
    flux_table = flux_table.rename(columns = {"ALMA flux": "es1_highz_alma", "Flux_err": "es1_highz_alma_err"})
    flux_table.to_csv(path + bcg + "/flux_table.csv", index = False)

In [12]:
from collections import Counter
other_bcgs = list((Counter(np.array(all_fluxes['id']))-Counter(high_z_es1)).elements())

In [13]:
for bcg in other_bcgs:
    flux_table = all_fluxes[all_fluxes['id'] == bcg]
    flux_table = flux_table.rename(columns = {"ALMA flux": bcg, "Flux_err": bcg + "_err"})
    flux_table.to_csv(path + bcg + "/flux_table.csv", index = False)

In [14]:
ms_es1 = ['ES1-12',
 'ES1-18',
 'ES1-25',
 'ES1-26',
 'ES1-34',
 'ES1-35']
ms_xmm = ['XMM-11',
 'XMM-19',
 'XMM-27',
 'XMM-29',
 'XMM-30']
ms_cdfs = ["CDFS-18", "CDFS-19"]

In [15]:
for bcg in ms_es1:
    flux_table = all_fluxes[all_fluxes['id'] == bcg]
    flux_table = flux_table.rename(columns = {"ALMA flux": "ES1-18", "Flux_err": "ES1-18_err"})
    flux_table.to_csv(path + bcg + "/flux_table.csv", index = False)
for bcg in ms_xmm:
    flux_table = all_fluxes[all_fluxes['id'] == bcg]
    flux_table = flux_table.rename(columns = {"ALMA flux": "XMM-11", "Flux_err": "XMM-11_err"})
    flux_table.to_csv(path + bcg + "/flux_table.csv", index = False)
for bcg in ms_cdfs:
    flux_table = all_fluxes[all_fluxes['id'] == bcg]
    flux_table = flux_table.rename(columns = {"ALMA flux": "CDFS-18", "Flux_err": "CDFS-18_err"})
    flux_table.to_csv(path + bcg + "/flux_table.csv", index = False)

In [16]:
all_fluxes[['id','redshift']].to_csv("/Users/arames52/Downloads/cigale-v2020.0/pcigale/data/bcg_redshifts.csv", index = False)

In [3]:
all_fluxes

NameError: name 'all_fluxes' is not defined